# Digital Elevation Model


## 1. Merge DEM from Individual Departments

Run this section to merge all the DEM files for the individual departments into one large for all of France


In [ ]:
import glob
import sys
from pyprojroot import here

sys.path.insert(0, "../../src")
from run_mp import *
from utilities import *

In [ ]:
# List all ascii files of the digital elevation model
all_files = glob.glob("../../data/raw/dem_france/all_files/*.asc")
print("Number of files: ", len(all_files))
display(all_files)

In [ ]:
# # See one file as example
# pd.read_csv(
#     "../../data/raw/dem_france/all_files/BDALTIV2_25M_FXX_0100_6800_MNT_LAMB93_IGN69.asc",
#     skiprows=6,
#     header=None,
#     delim_whitespace=True,
# )

# # data / raw / dem_france / all_files / BDALTIV2_25M_FXX_0075_6850_MNT_LAMB93_IGN69.asc.aux.xml

In [ ]:
# Combine all files into one
# ! Cell takes ca. 1 min to run
import glob
import rasterio
from rasterio.merge import merge
from rasterio.crs import CRS
from tqdm import tqdm

# List all .asc files in the directory
asc_files = glob.glob("../../data/raw/dem_france/all_files/*.asc")

# List for the data
src_files_to_mosaic = []

for file in tqdm(asc_files):
    src = rasterio.open(file)
    src_files_to_mosaic.append(src)

# Merge function returns a single mosaic array and the transformation info
mosaic, out_trans = merge(src_files_to_mosaic)

# Copy the metadata
out_meta = src.meta.copy()

# Update the metadata
out_meta.update(
    {
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "crs": CRS.from_epsg(2154).to_string(),
    }
)

In [ ]:
# Save final DEM as a tif file
with rasterio.open(
    here("data/raw/dem_france/original_dem25_altitude_put_together_in_python.tif"),
    "w",
    **out_meta
) as dest:
    dest.write(mosaic)

---


## 2. Produce derivatives in QGIS

See the batch process that I wrote to extract different variables like slope, aspect, etc. from the DEM at different scales. All done within QGIS.

Original file: `"data/raw/dem_france/original_dem25_altitude_put_together_in_python.tif"`


---


## 3. Extract data from raster files


In [ ]:
import glob
import sys
from pyprojroot import here

sys.path.insert(0, "../../src")
from run_mp import *
from utilities import *

In [ ]:
# Get files
files = glob.glob("/Volumes/SAMSUNG 1TB/qgis/france_dem/derivatives/*1000*.tif")
filenames = [f.split("/")[-1].split(".tif")[0] for f in files]
df_files = pd.DataFrame({"filename": filenames, "path": files})
list_files = split_df_into_list_of_group_or_ns(df_files, group_variable=files.__len__())

# Get buffered coordinates
buffer = gpd.read_file("../../data/final/nfi/700m_buffer_epsg2154.geojson")

# Run extraction
from utilities import extract_zonal_mean

run_mp(
    extract_zonal_mean,
    list_files,
    buffer=buffer,
    num_cores=10,
    force_run=True,
    save_dir="../../data/france_dem/zonal_statistics",
)

## 4. Clean raster extraction


In [ ]:
import glob
import sys
from pyprojroot import here

sys.path.insert(0, "../../src")
from run_mp import *
from utilities import *

In [ ]:
# Get all extracated files
files = glob.glob("../../data/france_dem/zonal_statistics/*.feather")

# Get structure of df
df_all = pd.read_feather(files[0])[["idp", "first_year"]]

# Loop through all files and attach to them by idp and first year
for file in files:
    df = pd.read_feather(file)
    df_all = pd.merge(df_all, df, on=["idp", "first_year"], how="left")

df_all

In [ ]:
# Aspect has a degree structure going from 0 to 360 which is not ideal for ML.
# We will transform it to a sin and cos variable
# https://stats.stackexchange.com/questions/218407/encoding-aspects-for-sine-and-cosine
df_fixed = df_all.copy()

# Extract variables that hold aspect information
aspect_vars = [var for var in df_fixed.columns if "aspect" in var]
print(aspect_vars)
wrangled_vars = 0

for var in aspect_vars:
    if "_sd" in var:
        df_fixed = df_fixed.drop(columns=[var])
        continue
    # Extract information on dem resolution
    res = var.split("_")[0]

    # Extract information on aspect type
    df_fixed[f"{res}_aspect_sin"] = np.sin(np.deg2rad(df_fixed[var]))
    df_fixed[f"{res}_aspect_cos"] = np.cos(np.deg2rad(df_fixed[var]))

    # Drop original variable
    df_fixed = df_fixed.drop(columns=[var])

    # Counter
    wrangled_vars += 1

# Verbose
print(f"Shape of df_fixed before aspect transformation:\t{df_all.shape}")
print(f"Shape of df_fixed after aspect transformation: \t{df_fixed.shape}")
print(f"Change in columns should be: {wrangled_vars*2-len(aspect_vars)}")

In [ ]:
df_fixed

In [ ]:
df_fixed.to_feather("../../data/final/predictor_datasets/topography.feather")

---
